In [1]:
import numpy as np
import pandas as pd
import pandas_profiling
import math

In [2]:
train = pd.read_csv('data/train.csv')

In [3]:

for col in ["TreatmentA", "TreatmentB", "TreatmentC", "TreatmentD", "stroke_in_2018"]:
    train[col] = pd.to_numeric(train[col],errors="coerce")
    train[col] = train[col].astype(int,errors='ignore')
    train[col] = train[col].apply(lambda x: x if x in [0,1] else None)

In [4]:
def merge(grp):
    df = pd.DataFrame()
    if(grp.shape[0] > 1):
        for c in grp.columns:
            value_counts = grp[c].value_counts().index.astype(grp[c].dtypes)
            if value_counts.size > 1:
                print(grp, value_counts) #Error
            elif value_counts.size == 1:
                df[c] = value_counts[0]
            else:
                df[c] = None
    else:
        df = grp.head(1)
    return df
train = train.groupby("id").apply(merge)

In [5]:
train["stroke_in_2018"].value_counts()

0.0    33947
1.0      648
Name: stroke_in_2018, dtype: int64

In [6]:
train = train[pd.notnull(train['stroke_in_2018'])]  #stroke_in_2018 is not null

In [7]:
train["TreatmentA"].value_counts()

0.0    1338
1.0     306
Name: TreatmentA, dtype: int64

In [8]:
train["TreatmentD"].value_counts()

0.0    1282
1.0     362
Name: TreatmentD, dtype: int64

In [9]:
# get the four colms
df = train[["TreatmentA", "TreatmentB", "TreatmentC", "TreatmentD", "stroke_in_2018"]]
# TreatementD turned out tobe always co-exist with NAN value stroke_in_2018

In [10]:
df.dropna(thresh=4,inplace=True) 

/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [11]:
df["TreatmentA"].value_counts()

0.0    1338
1.0     306
Name: TreatmentA, dtype: int64

In [12]:
df["TreatmentD"].value_counts()

0.0    1282
1.0     362
Name: TreatmentD, dtype: int64

In [ ]:
profile = df.profile_report(title='Treatement Effects Report')
profile.to_file(output_file="treatementEffects.html")

In [26]:
df.to_csv("treatment.csv",index=False)

In [14]:
from sklearn.preprocessing import PolynomialFeatures

In [22]:
poly = PolynomialFeatures(interaction_only=True,include_bias = False,degree=4)

In [23]:
X_train = df[["TreatmentA", "TreatmentB", "TreatmentC", "TreatmentD"]]
y_train = df["stroke_in_2018"]

In [24]:
X_train_interact = poly.fit_transform(X_train)

(1644, 15)